# Imports and Global Settings

In [1]:
import pandas as pd
from sqlalchemy import create_engine

pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

# Data Inbound

In [2]:
username = 'postgres'
password = ''
endpoint = ''
database = 'nba_betting'
port = '5432'

In [3]:
connection= create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

In [4]:
current_odds = pd.read_sql_table('dfc_covers_odds', connection)
current_game_results = pd.read_sql_table('dfc_covers_game_results', connection)
historic_odds = pd.read_sql_table('dfh_covers_odds', connection)
historic_standings = pd.read_sql_table('dfc_br_standings', connection)
historic_team_stats = pd.read_sql_table('dfc_br_team_stats', connection)
historic_opponent_stats = pd.read_sql_table('dfc_br_opponent_stats', connection)

# Standardize Dates

In [5]:
current_odds['datetime'] = pd.to_datetime(current_odds['date'])

In [6]:
current_game_results['datetime'] = pd.to_datetime(current_game_results['date'])

In [7]:
historic_odds['datetime'] = pd.to_datetime(historic_odds['date'])

In [8]:
historic_standings['datetime'] = pd.to_datetime(historic_standings['date'])

In [9]:
historic_team_stats['datetime'] = pd.to_datetime(historic_team_stats['date'])

In [10]:
historic_opponent_stats['datetime'] = pd.to_datetime(historic_opponent_stats['date'])

# Standardize Team Names

In [11]:
    team_full_name_map = {
        "Washington Wizards": "WAS",
        "Brooklyn Nets": "BKN",
        "Chicago Bulls": "CHI",
        "Miami Heat": "MIA",
        "Cleveland Cavaliers": "CLE",
        "Philadelphia 76ers": "PHI",
        "New York Knicks": "NYK",
        "Charlotte Hornets": "CHA",
        "Boston Celtics": "BOS",
        "Toronto Raptors": "TOR",
        "Milwaukee Bucks": "MIL",
        "Atlanta Hawks": "ATL",
        "Indiana Pacers": "IND",
        "Detroit Pistons": "DET",
        "Orlando Magic": "ORL",
        "Golden State Warriors": "GSW",
        "Phoenix Suns": "PHX",
        "Dallas Mavericks": "DAL",
        "Denver Nuggets": "DEN",
        "Los Angeles Clippers": "LAC",
        "Utah Jazz": "UTA",
        "Los Angeles Lakers": "LAL",
        "Memphis Grizzlies": "MEM",
        "Portland Trail Blazers": "POR",
        "Sacramento Kings": "SAC",
        "Oklahoma City Thunder": "OKC",
        "Minnesota Timberwolves": "MIN",
        "San Antonio Spurs": "SAS",
        "New Orleans Pelicans": "NOP",
        "Houston Rockets": "HOU",
        "Charlotte Bobcats": "CHA",
        "New Orleans Hornets": "NOP",
        "New Jersey Nets": "BKN",
        "Seattle SuperSonics": "OKC",
        "New Orleans/Oklahoma City Hornets": "NOP",
    }

    team_abrv_map = {
        "BK": "BKN",
        "BKN": "BKN",
        "BOS": "BOS",
        "MIL": "MIL",
        "ATL": "ATL",
        "CHA": "CHA",
        "CHI": "CHI",
        "CLE": "CLE",
        "DAL": "DAL",
        "DEN": "DEN",
        "DET": "DET",
        "GS": "GSW",
        "GSW": "GSW",
        "HOU": "HOU",
        "IND": "IND",
        "LAC": "LAC",
        "LAL": "LAL",
        "MEM": "MEM",
        "MIA": "MIA",
        "MIN": "MIN",
        "NO": "NOP",
        "NOP": "NOP",
        "NY": "NYK",
        "NYK": "NYK",
        "OKC": "OKC",
        "ORL": "ORL",
        "PHI": "PHI",
        "PHO": "PHX",
        "PHX": "PHX",
        "POR": "POR",
        "SA": "SAS",
        "SAS": "SAS",
        "SAC": "SAC",
        "TOR": "TOR",
        "UTA": "UTA",
        "WAS": "WAS",
    }

    team_short_name_map = {
        "Nets": "BKN",
        "Celtics": "BOS",
        "Bucks": "MIL",
        "Hawks": "ATL",
        "Hornets": "CHA",
        "Bulls": "CHI",
        "Cavaliers": "CLE",
        "Mavericks": "DAL",
        "Nuggets": "DEN",
        "Pistons": "DET",
        "Warriors": "GSW",
        "Rockets": "HOU",
        "Pacers": "IND",
        "Clippers": "LAC",
        "Lakers": "LAL",
        "Grizzlies": "MEM",
        "Heat": "MIA",
        "Timberwolves": "MIN",
        "Pelicans": "NOP",
        "Knicks": "NYK",
        "Thunder": "OKC",
        "Magic": "ORL",
        "76ers": "PHI",
        "Suns": "PHX",
        "Trail Blazers": "POR",
        "Spurs": "SAS",
        "Kings": "SAC",
        "Raptors": "TOR",
        "Jazz": "UTA",
        "Wizards": "WAS",
    }

    teams = dict(
        team_full_name_map.items()
        | team_abrv_map.items()
        | team_short_name_map.items()
    )

In [12]:
historic_standings['teamname'] = historic_standings['team'].map(teams)

In [13]:
historic_team_stats['teamname'] = historic_team_stats['team'].map(teams)

In [14]:
historic_opponent_stats['teamname'] = historic_opponent_stats['team'].map(teams)

In [15]:
current_odds['team'] = current_odds['home_team_short_name'].map(teams)

In [16]:
current_odds['opponent'] = current_odds['away_team_short_name'].map(teams)

In [17]:
current_game_results['team'] = current_game_results['home_team'].map(teams)

In [18]:
current_game_results['opponent'] = current_game_results['away_team'].map(teams)

# Combine Data

In [19]:
historic_odds['pred_datetime'] = historic_odds['datetime'] - pd.DateOffset(1)

In [20]:
current_odds['pred_datetime'] = current_odds['datetime'] - pd.DateOffset(1)

In [21]:
historic_df = historic_odds.merge(historic_standings,
                                   how='left',
                                   left_on=['team', 'pred_datetime'],
                                   right_on=['teamname', 'datetime'],
                                   suffixes=(None, '_sta'),
                                   validate='1:m')

In [22]:
historic_df = historic_df.merge(historic_standings,
                                   how='left',
                                   left_on=['opponent', 'pred_datetime'],
                                   right_on=['teamname', 'datetime'],
                                   suffixes=(None, '_osta'),
                                   validate='1:m')

In [23]:
historic_df = historic_df.merge(historic_team_stats,
                                   how='left',
                                   left_on=['team', 'pred_datetime'],
                                   right_on=['teamname', 'datetime'],
                                   suffixes=(None, '_ts'),
                                   validate='1:m')

In [24]:
historic_df = historic_df.merge(historic_opponent_stats,
                                   how='left',
                                   left_on=['opponent', 'pred_datetime'],
                                   right_on=['teamname', 'datetime'],
                                   suffixes=(None, '_os'),
                                   validate='1:m')

In [25]:
side_df = current_odds.merge(current_game_results,
                                   how='left',
                                   left_on=['team', 'datetime'],
                                   right_on=['team', 'datetime'],
                                   suffixes=(None, '_gr'),
                                   validate='1:1')

In [26]:
side_df = side_df.merge(historic_standings,
                                   how='left',
                                   left_on=['team', 'pred_datetime'],
                                   right_on=['teamname', 'datetime'],
                                   suffixes=(None, '_sta'),
                                   validate='1:m')

In [27]:
side_df = side_df.merge(historic_standings,
                                   how='left',
                                   left_on=['opponent', 'pred_datetime'],
                                   right_on=['teamname', 'datetime'],
                                   suffixes=(None, '_osta'),
                                   validate='1:m')

In [28]:
side_df = side_df.merge(historic_team_stats,
                                   how='left',
                                   left_on=['team', 'pred_datetime'],
                                   right_on=['teamname', 'datetime'],
                                   suffixes=(None, '_ts'),
                                   validate='1:m')

In [29]:
side_df = side_df.merge(historic_opponent_stats,
                                   how='left',
                                   left_on=['opponent', 'pred_datetime'],
                                   right_on=['teamname', 'datetime'],
                                   suffixes=(None, '_os'),
                                   validate='1:m')

# Select Dates

In [30]:
historic_df = historic_df[historic_df['datetime'] >= '2022-02-27']

In [31]:
side_df = side_df[side_df['datetime'].between('2022-05-12', '2022-05-21', inclusive=False)]

# Concat Dataframes

In [32]:
side_df = side_df.drop(columns=['time', 'home_team_full_name',
                                'away_team_full_name', 'open_line_away'])

In [33]:
side_df = side_df.rename(columns={'id_num': 'id',
                                    'home_score': 'score',
                                    'away_score': 'opponent_score',
                                    'link': 'game_url',
                                    'open_line_home': 'spread'})

In [34]:
side_df['result'] = side_df.apply(lambda x: 'W' if x['score'] > x['opponent_score'] else 'L', axis=1)

In [35]:
side_df['spread_result'] = side_df.apply(lambda x: 'W' if (x['score'] - x['opponent_score']) > -x['spread'] else 'L', axis=1)

In [36]:
full_dataset = pd.concat((side_df, historic_df), ignore_index=True)

# Create Unique Record ID

In [37]:
full_dataset['id'] = full_dataset['datetime'].apply(lambda x: x.strftime('%Y%m%d')) + full_dataset['team'] + full_dataset['opponent']

# Cleanup

## Remove Excess Columns and Reorder

In [38]:
columns_to_keep = ['id', 'datetime', 'team', 'opponent', 'score', 'opponent_score', 'result',
                   'game_url', 'league_year', 'spread', 'spread_result', 'pred_datetime',
                   'wins', 'losses', 'win_perc', 'expected_wins', 'expected_losses',
                   'points_scored_per_game', 'points_allowed_per_game',
                   'wins_osta', 'losses_osta', 'win_perc_osta', 'expected_wins_osta', 'expected_losses_osta',
                   'points_scored_per_game_osta', 'points_allowed_per_game_osta',
                   'g', 'mp', 'pts', 'ast', 'trb', 'blk', 'stl',
                   'tov', 'pf', 'drb', 'orb',
                   'fg', 'fga', 'fg_pct', 'fg2', 'fg2a', 'fg2_pct',
                   'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct',
                   'opp_g', 'opp_mp', 'opp_pts', 'opp_ast', 'opp_trb', 'opp_blk', 'opp_stl',
                   'opp_tov', 'opp_pf', 'opp_drb', 'opp_orb',
                   'opp_fg', 'opp_fga', 'opp_fg_pct', 'opp_fg2', 'opp_fg2a', 'opp_fg2_pct',
                   'opp_fg3', 'opp_fg3a', 'opp_fg3_pct', 'opp_ft', 'opp_fta', 'opp_ft_pct']

In [39]:
full_dataset = full_dataset[columns_to_keep]

## Rename Columns

In [40]:
column_rename_dict = {'id': 'game_id', 'datetime': 'game_date', 'team': 'home_team', 'opponent': 'away_team',
                      'score': 'home_score', 'opponent_score': 'away_score', 'result': 'home_result',
                      'game_url': 'covers_game_url', 'league_year': 'league_year', 'spread': 'home_spread',
                      'spread_result': 'home_spread_result', 'pred_datetime': 'pred_date',
                      'wins': 'wins', 'losses': 'losses', 'win_perc': 'win_pct',
                      'expected_wins': 'expected_wins', 'expected_losses': 'expected_losses',
                      'points_scored_per_game': 'home_ppg', 'points_allowed_per_game': 'home_papg',
                      'wins_osta': 'away_wins', 'losses_osta': 'away_losses', 'win_perc_osta': 'away_win_pct',
                      'expected_wins_osta': 'away_expected_wins', 'expected_losses_osta': 'away_expected_losses',
                      'points_scored_per_game_osta': 'away_ppg', 'points_allowed_per_game_osta': 'away_papg',
                      'g': 'g', 'mp': 'mp', 'pts': 'pts', 'ast': 'ast', 'trb': 'trb', 'blk': 'blk',
                      'stl': 'stl', 'tov': 'tov', 'pf': 'pf', 'drb': 'drb', 'orb': 'orb',
                      'fg': 'fg', 'fga': 'fga', 'fg_pct': 'fg_pct',
                      'fg2': 'fg2', 'fg2a': 'fg2a', 'fg2_pct': 'fg2_pct',
                      'fg3': 'fg3', 'fg3a': 'fg3a', 'fg3_pct': 'fg3_pct',
                      'ft': 'ft', 'fta': 'fta', 'ft_pct': 'ft_pct',
                      'opp_g': 'away_g', 'opp_mp': 'away_mp', 'opp_pts': 'away_pts', 'opp_ast': 'away_ast',
                      'opp_trb': 'away_trb', 'opp_blk': 'away_blk', 'opp_stl': 'away_stl',
                      'opp_tov': 'away_tov', 'opp_pf': 'away_pf', 'opp_drb': 'away_drb', 'opp_orb': 'away_orb',
                      'opp_fg': 'away_fg', 'opp_fga': 'away_fga', 'opp_fg_pct': 'away_fg_pct',
                      'opp_fg2': 'away_fg2', 'opp_fg2a': 'away_fg2a', 'opp_fg2_pct': 'away_fg2_pct',
                      'opp_fg3': 'away_fg3', 'opp_fg3a': 'away_fg3a', 'opp_fg3_pct': 'away_fg3_pct',
                      'opp_ft': 'away_ft', 'opp_fta': 'away_fta', 'opp_ft_pct': 'away_fta_pct'}

In [41]:
full_dataset = full_dataset.rename(columns=column_rename_dict)

# Review Data to be Saved

In [42]:
full_dataset.sort_values('game_id', ascending=True).head(10)

,game_id,game_date,home_team,away_team,home_score,away_score,home_result,covers_game_url,league_year,home_spread,home_spread_result,pred_date,wins,losses,win_pct,expected_wins,expected_losses,home_ppg,home_papg,away_wins,away_losses,away_win_pct,away_expected_wins,away_expected_losses,away_ppg,away_papg,g,mp,pts,ast,trb,blk,stl,tov,pf,drb,orb,fg,fga,fg_pct,fg2,fg2a,fg2_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,away_g,away_mp,away_pts,away_ast,away_trb,away_blk,away_stl,away_tov,away_pf,away_drb,away_orb,away_fg,away_fga,away_fg_pct,away_fg2,away_fg2a,away_fg2_pct,away_fg3,away_fg3a,away_fg3_pct,away_ft,away_fta,away_fta_pct
282,20220227CHADET,2022-02-27,CHA,DET,126.0,127.0,L,/sport/basketball/nba/boxscore/251468,2021-2022,-10.0,L,2022-02-26,30.0,31.0,0.492,30.0,31.0,114.0,114.4,14.0,46.0,0.233,13.0,47.0,102.6,112.5,61.0,14815.0,6954.0,1664.0,2760.0,297.0,531.0,789.0,1219.0,2084.0,676.0,2580.0,5633.0,0.458,1744.0,3287.0,0.531,836.0,2346.0,0.356,958.0,1301.0,0.736,60.0,14450.0,6747.0,1534.0,2793.0,310.0,452.0,855.0,1201.0,2186.0,607.0,2448.0,5152.0,0.475,1708.0,3175.0,0.538,740.0,1977.0,0.374,1111.0,1438.0,0.773
315,20220227GSWDAL,2022-02-27,GSW,DAL,101.0,107.0,L,/sport/basketball/nba/boxscore/251470,2021-2022,-3.5,L,2022-02-26,43.0,17.0,0.717,43.0,17.0,111.0,103.6,35.0,25.0,0.583,36.0,24.0,106.7,103.5,60.0,14450.0,6658.0,1641.0,2747.0,291.0,558.0,872.0,1223.0,2164.0,583.0,2422.0,5185.0,0.467,1551.0,2777.0,0.559,871.0,2408.0,0.362,943.0,1232.0,0.765,60.0,14475.0,6208.0,1394.0,2653.0,226.0,409.0,789.0,1196.0,2089.0,564.0,2325.0,5101.0,0.456,1684.0,3197.0,0.527,641.0,1904.0,0.337,917.0,1213.0,0.756
357,20220227HOULAC,2022-02-27,HOU,LAC,98.0,99.0,L,/sport/basketball/nba/boxscore/251469,2021-2022,6.5,W,2022-02-26,15.0,44.0,0.254,14.0,45.0,108.5,118.2,31.0,31.0,0.500,29.0,33.0,106.8,107.6,59.0,14185.0,6403.0,1357.0,2497.0,272.0,429.0,957.0,1199.0,1919.0,578.0,2287.0,5070.0,0.451,1504.0,2784.0,0.540,783.0,2286.0,0.343,1046.0,1468.0,0.713,62.0,14930.0,6671.0,1478.0,2948.0,261.0,490.0,835.0,1154.0,2200.0,748.0,2514.0,5628.0,0.447,1800.0,3506.0,0.513,714.0,2122.0,0.336,929.0,1182.0,0.786
247,20220227INDBOS,2022-02-27,IND,BOS,128.0,107.0,W,/sport/basketball/nba/boxscore/251467,2021-2022,9.5,W,2022-02-26,20.0,41.0,0.328,25.0,36.0,109.6,112.2,36.0,26.0,0.581,42.0,20.0,109.2,103.5,61.0,14815.0,6683.0,1503.0,2723.0,340.0,424.0,818.0,1243.0,2045.0,678.0,2493.0,5443.0,0.458,1773.0,3296.0,0.538,720.0,2147.0,0.335,977.0,1289.0,0.758,62.0,15080.0,6417.0,1338.0,2763.0,285.0,468.0,809.0,1191.0,2112.0,651.0,2348.0,5474.0,0.429,1622.0,3315.0,0.489,726.0,2159.0,0.336,995.0,1275.0,0.780
337,20220227LALNOP,2022-02-27,LAL,NOP,95.0,123.0,L,/sport/basketball/nba/boxscore/251472,2021-2022,-2.5,L,2022-02-26,27.0,32.0,0.458,27.0,32.0,110.8,112.3,24.0,36.0,0.400,24.0,36.0,106.5,109.4,59.0,14385.0,6535.0,1423.0,2644.0,335.0,472.0,826.0,1183.0,2074.0,570.0,2441.0,5247.0,0.465,1737.0,3221.0,0.539,704.0,2026.0,0.347,949.0,1316.0,0.721,60.0,14450.0,6565.0,1434.0,2534.0,298.0,456.0,770.0,1221.0,1974.0,560.0,2395.0,5146.0,0.465,1623.0,3019.0,0.538,772.0,2127.0,0.363,1003.0,1281.0,0.783
224,20220227NYKPHI,2022-02-27,NYK,PHI,109.0,125.0,L,/sport/basketball/nba/boxscore/251465,2021-2022,7.5,L,2022-02-26,25.0,35.0,0.417,27.0,33.0,105.0,106.7,36.0,23.0,0.610,33.0,26.0,107.9,105.9,60.0,14500.0,6298.0,1289.0,2763.0,299.0,413.0,771.0,1205.0,2081.0,682.0,2245.0,5178.0,0.434,1461.0,2952.0,0.495,784.0,2226.0,0.352,1024.0,1364.0,0.751,59.0,14260.0,6248.0,1365.0,2657.0,272.0,374.0,738.0,1126.0,2041.0,616.0,2313.0,5132.0,0.451,1649.0,3164.0,0.521,664.0,1968.0,0.337,958.0,1229.0,0.779
317,20220227PHXUTA,2022-02-27,PHX,UTA,114.0,118.0,L,/sport/basketball/nba/boxscore/251466,2021-2022,3.0,L,2022-02-26,49.0,11.0,0.817,44.0,16.0,113.6,105.7,37.0,22.0,0.627,42.0,17.0,113.7,106.9,60.0,14425.0,6818.0,1623.0,2759.0,260.0,515.0,750.0,1153.0,2162.0,597.0,2588.0,5372.0,0.482,1909.0,3481.0,0.548,679.0,1891.0,0.359,963.0,1215.0,0.793,59.0,14160.0,6308.0,1360.0,2469.0,248.0,450.0,676.0,1183.0,1873.0,596

In [43]:
full_dataset[full_dataset.isna().any(axis=1)]

,game_id,game_date,home_team,away_team,home_score,away_score,home_result,covers_game_url,league_year,home_spread,home_spread_result,pred_date,wins,losses,win_pct,expected_wins,expected_losses,home_ppg,home_papg,away_wins,away_losses,away_win_pct,away_expected_wins,away_expected_losses,away_ppg,away_papg,g,mp,pts,ast,trb,blk,stl,tov,pf,drb,orb,fg,fga,fg_pct,fg2,fg2a,fg2_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,away_g,away_mp,away_pts,away_ast,away_trb,away_blk,away_stl,away_tov,away_pf,away_drb,away_orb,away_fg,away_fga,away_fg_pct,away_fg2,away_fg2a,away_fg2_pct,away_fg3,away_fg3a,away_fg3_pct,away_ft,away_fta,away_fta_pct
3,20220517MIABOS,2022-05-17,MIA,BOS,118.0,107.0,W,/sport/basketball/nba/matchup/265572,2021-2022,-2.0,W,2022-05-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,19905.0,8567.0,1773.0,3585.0,376.0,618.0,1065.0,1592.0,2727.0,858.0,3137.0,7221.0,0.434,2168.0,4361.0,0.497,969.0,2860.0,0.339,1324.0,1692.0,0.783
41,20220509MILBOS,2022-05-09,MIL,BOS,108.0,116.0,L,/sport/basketball/nba/boxscore/265147,2021-2022,-1.5,L,2022-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,19905.0,8567.0,1773.0,3585.0,376.0,618.0,1065.0,1592.0,2727.0,858.0,3137.0,7221.0,0.434,2168.0,4361.0,0.497,969.0,2860.0,0.339,1324.0,1692.0,0.783
59,20220509GSWMEM,2022-05-09,GSW,MEM,101.0,98.0,W,/sport/basketball/nba/boxscore/265177,2021-2022,-9.5,L,2022-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,19780.0,9014.0,2047.0,3527.0,496.0,588.0,1180.0,1625.0,2706.0,821.0,3305.0,7269.0,0.455,2318.0,4441.0,0.522,987.0,2828.0,0.349,1417.0,1837.0,0.771


# Save to RDS

In [44]:
# full_dataset.to_sql(name='combined_data_inbound', con=connection, index=False, if_exists='append')